In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from src.models.net import Net
from src.models.onboard_dataset import OnboardDataset

In [22]:
class TrainModel():
    def __init__(self, train_set="./data/processed/train_set.npz", 
                 valid_set="./data/processed/valid_set.npz"):    
        no_cuda = True
        use_cuda = not no_cuda and torch.cuda.is_available()
        torch.manual_seed(1)
        self.device = torch.device("cuda" if use_cuda else "cpu")
        kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

        train_batch_size = 64
        valid_batch_size = 100
        lr = 0.01
        momentum = 0.5
        
        self.log_interval = 100
        self.model = Net().to(self.device)
        self.optimizer = optim.SGD(self.model.parameters(), lr=lr, momentum=momentum)
        self.loss = nn.CrossEntropyLoss()   
        
        mnist_transform = transforms.Compose([
            transforms.ToTensor(), 
            transforms.Normalize((0.1307,), (0.3081,))
        ])             
        self.train_loader = torch.utils.data.DataLoader(
                OnboardDataset(path=train_set, transform=mnist_transform),
                batch_size = train_batch_size, shuffle=True, **kwargs)
        self.valid_loader = torch.utils.data.DataLoader(
                OnboardDataset(path=valid_set, transform=mnist_transform),
                batch_size = valid_batch_size, shuffle=True, **kwargs)
        
    def train_model(self, epoch):
        self.model.train()
        for batch_idx, (data, target) in enumerate(self.train_loader):
            data, target = data.to(self.device), target.to(self.device)
            self.optimizer.zero_grad()
            output = self.model(data)
            loss = self.loss(output, target)
            loss.backward()
            self.optimizer.step()
            if batch_idx % self.log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(self.train_loader.dataset),
                    100. * batch_idx / len(self.train_loader), loss.item()))

    def valid_model(self):
        self.model.eval()
        val_loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in self.valid_loader:
                data, target = data.to(self.device), target.to(self.device)
                output = self.model(data)
                val_loss += self.loss(output, target).item() 
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        val_loss /= len(self.valid_loader.dataset)

        print('\nValid set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format
              (val_loss, correct, len(self.valid_loader.dataset),
            100. * correct / len(self.valid_loader.dataset)))
        
    def train(self, epochs, save_model_path="./models/model.pt"):
        for epoch in range(epochs):
            self.train_model(epoch)
            self.valid_model()
        torch.save(self.model, save_model_path)         

In [23]:
if __name__ == "__main__":
    train_model = TrainModel(train_set="./data/processed/train_set.npz", 
                            valid_set="./data/processed/valid_set.npz", )
    train_model.train(epochs=2, save_model_path="./models/model.pt")


Train Epoch: 0 [0/50000 (0%)]	Loss: 2.311310
Train Epoch: 0 [6400/50000 (13%)]	Loss: 0.670621
Train Epoch: 0 [12800/50000 (26%)]	Loss: 0.739158
Train Epoch: 0 [19200/50000 (38%)]	Loss: 0.386902
Train Epoch: 0 [25600/50000 (51%)]	Loss: 0.371745
Train Epoch: 0 [32000/50000 (64%)]	Loss: 0.494126
Train Epoch: 0 [38400/50000 (77%)]	Loss: 0.281415
Train Epoch: 0 [44800/50000 (90%)]	Loss: 0.243157

Valid set: Average loss: 0.0027, Accuracy: 9208/10000 (92%)

Train Epoch: 1 [0/50000 (0%)]	Loss: 0.226171
Train Epoch: 1 [6400/50000 (13%)]	Loss: 0.179414
Train Epoch: 1 [12800/50000 (26%)]	Loss: 0.255594
Train Epoch: 1 [19200/50000 (38%)]	Loss: 0.366122
Train Epoch: 1 [25600/50000 (51%)]	Loss: 0.227165
Train Epoch: 1 [32000/50000 (64%)]	Loss: 0.378966
Train Epoch: 1 [38400/50000 (77%)]	Loss: 0.461857
Train Epoch: 1 [44800/50000 (90%)]	Loss: 0.496010

Valid set: Average loss: 0.0022, Accuracy: 9377/10000 (94%)



In [9]:
model = torch.load(PATH)
model.eval()

Net(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)